In [11]:
from spektral.layers import GCNConv

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import tensorflow as tf
from tensorflow.keras.regularizers import l2
import pandas as pd

In [12]:
import numpy as np
import scipy.sparse as sp
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, BatchNormalization
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.data import Dataset, DisjointLoader, Graph
from spektral.layers import GCSConv, GlobalAvgPool, ChebConv, GlobalSumPool
from spektral.layers.pooling import TopKPool
from spektral.transforms.normalize_adj import NormalizeAdj
import pickle as pkl
import networkx as nx
import pdb
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [13]:
################################################################################
# PARAMETERS
################################################################################
learning_rate = 1e-2  # Learning rate
epochs = 1000  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 3  # Batch size


################################################################################
# LOAD DATA
################################################################################
import sys
from spektral.data import Dataset, Graph
# sys.path.append('../lib')
# from data_pre_processing import load_data
# sys.path.remove('../lib')
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import AdjToSpTensor, LayerPreprocess
from sklearn.preprocessing import MinMaxScaler


x_data = []
y_data = []

def load_data(circuit_name, path_to_data="data", normalize=False):
    """Load data."""
    names = ["x", "y", "graph"]
    objects = []
    for i in range(len(names)):
        with open(f"{path_to_data}/{circuit_name}.{names[i]}", "rb") as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.load(f, encoding="latin1"))
            else:
                objects.append(pkl.load(f))

    x, y, graph = tuple(objects)
    
    x = np.array(x).astype("float32")
    for x_feat in x:
        x_data.append(x_feat)
        
    features = sp.csr_matrix(x).astype('float32')
    # adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph)).astype(int)
    g = nx.DiGraph()
    g.add_nodes_from(graph.keys())
    for k, v in graph.items():
      g.add_edges_from(([(k, t) for t in v]))
      g.add_edges_from([(k, k)])
    adj = nx.adjacency_matrix(g)
    labels = np.array(y).astype('float32').reshape((-1,1))
    for l in y:
        y_data.append(l)
    print(adj.shape)
    print(features.shape)
    return adj, features, labels
  
def encode_label(labels):
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)
    labels = to_categorical(labels)
    return labels


def _preprocess_features(features):
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.0
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return features



class CircuitDataset(Dataset):
    def read(self):
        circuits = []
#         circs = ['c6288','c5315','c432', 'c499', 'c17', 'c880', 'c1355', 'c1908', 'c3540', 'adder.bench', 'arbiter.bench', 'cavlc.bench', 'dec.bench', 'voter.bench', "sin.bench","priority.bench", "multiplier.bench", "max.bench"]
        circs = ["adder.bench","arbiter.bench","c1355","c1908","c3540","c432","c499","c5315","c6288","c880","cavlc.bench","dec.bench","int2float.bench","max.bench","multiplier.bench","priority.bench","sin.bench","voter.bench"]
#         circs = ["log2.bench"]
#         circs = ['adder.bench', "arbiter.bench",  "sin.bench", "multiplier.bench", "voter.bench", "priority.bench"]
        for circ in circs:
            A, X, labels = load_data(circ, '../data/output', normalize="")
#             if sum(labels) >= 500:
            print(f"{circ}: {sum(labels)}, {len(labels)}")
            circuits.append(Graph(x=X.toarray(), a=A, y=labels))

        return circuits

def normalize_feature(circ_dataset):
    scaler = MinMaxScaler()
    scaler.fit(x_data)
    for graph in circ_dataset:
        graph.x = scaler.transform(graph.x)
    return circ_dataset

dataset = normalize_feature(CircuitDataset(transforms=[LayerPreprocess(ChebConv)]))

# Parameters

F = dataset.n_node_features  # Dimension of node features
n_out = dataset.n_labels  # Dimension of the target

# Train/valid/test split
np.random.seed(42)
idxs = np.random.permutation(len(dataset))
split_va, split_te = int(0.8 * len(dataset)), int(0.9 * len(dataset))
idx_tr, idx_va, idx_te = np.split(idxs, [split_va, split_te])
print(idx_tr, idx_va, idx_te)
dataset_tr = dataset[idx_tr]
dataset_va = dataset[idx_va]
dataset_te = dataset[idx_te]
# dataset_tr = dataset[[ 0,7, 8,  14,  3, 13, 10, 15, 11,  2,  9, 17,  4, 12]]
# dataset_va = dataset[[1, 6]]
# dataset_te = dataset[[16, 5]]
# dataset_tr = dataset[[0,1]]
# dataset_va = dataset[[2, 3]]
# dataset_te = dataset[[4, 5]]

# loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs, node_level=True)
# loader_va = DisjointLoader(dataset_va, batch_size=batch_size, node_level=True)
# loader_te = DisjointLoader(dataset_te, batch_size=batch_size, node_level=True)
def load_tr_data(epochs=400):
    loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs, node_level=True)
    loader_va = DisjointLoader(dataset_va, batch_size=batch_size, node_level=True)
    loader_te = DisjointLoader(dataset_te, batch_size=batch_size, node_level=True)
    return loader_tr, loader_va, loader_te

loader_tr, loader_va, loader_te = load_tr_data()


(4209, 4209)
(4209, 4)
adder.bench: [637.], 4209
(36352, 36352)
(36352, 4)
arbiter.bench: [3763.], 36352
(1355, 1355)
(1355, 4)
c1355: [192.], 1355
(1908, 1908)
(1908, 4)
c1908: [257.], 1908
(3540, 3540)
(3540, 4)
c3540: [406.], 3540
(432, 432)
(432, 4)
c432: [60.], 432
(499, 499)
(499, 4)
c499: [50.], 499
(5315, 5315)
(5315, 4)
c5315: [599.], 5315
(6288, 6288)
(6288, 4)
c6288: [846.], 6288
(880, 880)
(880, 4)
c880: [114.], 880
(2164, 2164)
(2164, 4)
cavlc.bench: [318.], 2164
(936, 936)
(936, 4)
dec.bench: [186.], 936
(761, 761)
(761, 4)
int2float.bench: [80.], 761
(9996, 9996)
(9996, 4)
max.bench: [1040.], 9996
(89943, 89943)
(89943, 4)
multiplier.bench: [9294.], 89943
(3495, 3495)
(3495, 4)
priority.bench: [350.], 3495
(16771, 16771)
(16771, 4)
sin.bench: [1723.], 16771
(50122, 50122)
(50122, 4)
voter.bench: [5012.], 50122
[ 0  1  8  5  3 13 16 15 11  2  9 17  4 12] [ 7 10] [14  6]


In [14]:
N = sum([ g.n_nodes for g in dataset.graphs ])
n_out = 1
N

234966

In [15]:
from sklearn.utils.class_weight import compute_class_weight

y_data = []
for data_tr in dataset_tr:
    y_data.append(data_tr.y)
y_data = np.vstack((y_data)).reshape((-1,))


def _compute_class_weight_dictionary(y):
    # helper for returning a dictionary instead of an array
    classes = np.unique(y).astype('float32')
    class_weight = compute_class_weight("balanced", classes, y)
    class_weight_dict = dict(zip(classes, class_weight))
    return class_weight_dict 

weights = _compute_class_weight_dictionary(np.vstack((y_data)).reshape((-1,)))
print(weights)

{0.0: 0.559920411181657, 1.0: 4.672201009136779}


/Users/nikitaacharya/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass classes=[0. 1.], y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [16]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [17]:
from focal_loss import BinaryFocalLoss
from tensorflow.keras.metrics import sparse_categorical_accuracy, binary_accuracy, binary_crossentropy

focal_loss_fn = BinaryFocalLoss(gamma=0.9999, pos_weight=1)

In [18]:
from tensorflow.keras.metrics import sparse_categorical_accuracy, binary_accuracy, binary_crossentropy
from tensorflow.keras import activations
from tensorflow.keras import layers

channels = 256         # Number of channels in the first layer
dropout = 0.05         # Dropout rate for the features
l2_reg = 5e-4           # L2 regularization rate
learning_rate = 1e-3    # Learning rate
epochs = 500            # Number of training epochs
es_patience = 10    
patience = 100 # Patience for early stopping
# N = 

# Model definition
X_in = Input(shape=(F, ))
print("X_in: ", X_in.shape)
fltr_in = Input((None, ), sparse=True)
mask_in = Input((None, ))

# x_1 = BatchNormalization()(X_in)
# dropout_1 = Dropout(dropout)(x_1)
use_bias = True
x = ChebConv(512, K=2,
                         dropout_rate=dropout,
                         activation='relu',
                         kernel_regularizer=l2(l2_reg),
                         attn_kernel_regularizer=l2(l2_reg),
                         bias_regularizer=l2(l2_reg),
                         use_bias=use_bias,
                         n_input_channels=dataset.n_node_features, name="conv1")([X_in, fltr_in])
x = BatchNormalization()(x)
x = Dropout(dropout)(x)
x = ChebConv(256, K=2,
                         dropout_rate=dropout,
                         activation='relu',
                         kernel_regularizer=l2(l2_reg),
                         attn_kernel_regularizer=l2(l2_reg),
                         bias_regularizer=l2(l2_reg),
                         use_bias=use_bias,
                         n_input_channels=dataset.n_node_features, name="conv2")([X_in, fltr_in])
x = BatchNormalization()(x)
x = Dropout(dropout)(x)
x = ChebConv(128, K=2,
                         dropout_rate=dropout,
                         activation='relu',
                         kernel_regularizer=l2(l2_reg),
                         attn_kernel_regularizer=l2(l2_reg),
                         bias_regularizer=l2(l2_reg),
                         use_bias=use_bias,
                         n_input_channels=dataset.n_node_features, name="conv3")([x, fltr_in])
x = BatchNormalization()(x)
x = Dropout(dropout)(x)
x = ChebConv(32, K=2,
                         dropout_rate=dropout,
                         activation='relu',
                         kernel_regularizer=l2(l2_reg),
                         attn_kernel_regularizer=l2(l2_reg),
                         bias_regularizer=l2(l2_reg),
                         use_bias=use_bias,
                         n_input_channels=dataset.n_node_features, name="conv4")([x, fltr_in])
x = BatchNormalization()(x)
x = Dropout(dropout)(x)
x = ChebConv(16, K=2,
                         dropout_rate=dropout,
                         activation='relu',
                         kernel_regularizer=l2(l2_reg),
                         attn_kernel_regularizer=l2(l2_reg),
                         bias_regularizer=l2(l2_reg),
                         use_bias=use_bias,
                         n_input_channels=dataset.n_node_features, name="conv5")([x, fltr_in])
x = BatchNormalization()(x)
x = Dropout(dropout)(x)
x = ChebConv(8, K=2,
                         dropout_rate=dropout,
                         activation='relu',
                         kernel_regularizer=l2(l2_reg),
                         attn_kernel_regularizer=l2(l2_reg),
                         bias_regularizer=l2(l2_reg),
                         use_bias=use_bias,
                         n_input_channels=dataset.n_node_features, name="conv6")([x, fltr_in])

x = BatchNormalization()(x)
x = Dropout(dropout)(x)
x = ChebConv(4, K=2,
                         dropout_rate=dropout,
                         activation='relu',
                         kernel_regularizer=l2(l2_reg),
                         attn_kernel_regularizer=l2(l2_reg),
                         bias_regularizer=l2(l2_reg),
                         use_bias=use_bias,
                         n_input_channels=dataset.n_node_features, name="conv7")([x, fltr_in])
x = BatchNormalization()(x)
x = Dropout(dropout)(x)
x = ChebConv(n_out, activation='sigmoid',
                         n_input_channels=dataset.n_node_features, name="conv8", use_bias=use_bias)([x, fltr_in])

model = Model(inputs=[X_in, fltr_in, mask_in], outputs=x)
optimizer = Adam(lr=learning_rate)

def masked_loss_and_accuracy(target, predictions):
    loss = binary_crossentropy(target, predictions)
    minimum = tf.math.minimum(tf.reduce_sum(tf.dtypes.cast(target == 0, tf.int32)), tf.reduce_sum(tf.dtypes.cast(target == 1, tf.int32)))
    zeroes_index = tf.random.shuffle(tf.where(target == 0)[:,0])
    ones_index = tf.random.shuffle(tf.where(target == 1)[:,0])
    loss_zeroes =  tf.reduce_mean(tf.gather(loss, zeroes_index))
    loss_ones = tf.reduce_mean(tf.gather(loss, ones_index))
    
    weight_for_0 = tf.cast((1 / tf.shape(zeroes_index)[0])*(tf.cast(tf.shape(target)[0], 'float64'))/2.0, 'float32')
    weight_for_1 = tf.cast((1 / tf.shape(ones_index)[0])*(tf.cast(tf.shape(target)[0], 'float64'))/2.0, 'float32')

    return (weight_for_0 * loss_zeroes * 0.6 +  weight_for_1 * loss_ones * 0.4)


model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              weighted_metrics=['acc', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
model.summary()

tbCallBack_GCN = tf.keras.callbacks.TensorBoard(
        log_dir='logs/WorkingModelWithBiasGCN1',
        histogram_freq=1
)
# EarlyStopping(patience=patience, restore_best_weights=True)
callback_GCN = [tbCallBack_GCN]

X_in:  (None, 4)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
conv2 (ChebConv)                (None, 256)          2304        input_4[0][0]                    
                                                                 input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 256)          1024        conv2[0][0

In [ ]:
# channels = 256  # Number of channels for GCN layers
# dropout = 0.5  # Dropout rate for the features
# learning_rate = 1e-3  # Learning rate
# epochs = 200 

# x_in = Input(shape=(F,))
# a_in = Input((None,), sparse=True)
# mask_in = Input((None, ))

# x_1 = GCNConv(channels, activation="relu")([x_in, a_in])
# x_1 = BatchNormalization()(x_1)
# x_1 = Dropout(dropout)(x_1)
# x_2 = GCNConv(channels, activation="relu")([x_1, a_in])
# x_2 = BatchNormalization()(x_2)
# x_2 = Dropout(dropout)(x_2)
# fc2 = GCNConv(n_out, activation="sigmoid")(x_2)  


# model = Model(inputs=[x_in, a_in, mask_in], outputs=fc2)
# optimizer = Adam(lr=learning_rate)
# model.compile(optimizer=optimizer,
#               loss='binary_crossentropy',
#               weighted_metrics=['acc', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
# model.summary()

# tbCallBack_GCN = tf.keras.callbacks.TensorBoard(
#         log_dir='logs/WorkingModelNobias12345678912',
#         histogram_freq=1
# )
# callback_GCN = [tbCallBack_GCN]

In [19]:
epochs = 1000
loader_tr, loader_va, loader_te = load_tr_data(epochs)
history = model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
    class_weight=weights,
    callbacks=callback_GCN
)

Epoch 1/1000
5/5 [==============================] - 5s 471ms/step - loss: 1.0105 - acc: 0.4746 - precision_1: 0.4573 - recall_1: 0.3221 - val_loss: 0.8208 - val_acc: 0.8612 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/1000
5/5 [==============================] - 2s 363ms/step - loss: 0.7810 - acc: 0.6796 - precision_1: 0.6735 - recall_1: 0.7081 - val_loss: 0.8148 - val_acc: 0.8774 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/1000
5/5 [==============================] - 2s 412ms/step - loss: 0.7719 - acc: 0.6896 - precision_1: 0.6818 - recall_1: 0.7372 - val_loss: 0.8040 - val_acc: 0.8774 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 4/1000
5/5 [==============================] - 2s 412ms/step - loss: 0.6983 - acc: 0.7470 - precision_1: 0.7308 - recall_1: 0.7821 - val_loss: 0.7912 - val_acc: 0.8774 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 5/1000
5/5 [==============================] - 2s 419ms/step - loss: 0.6667

5/5 [==============================] - 2s 375ms/step - loss: 0.5373 - acc: 0.8346 - precision_1: 0.8249 - recall_1: 0.8515 - val_loss: 0.6116 - val_acc: 0.8774 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 37/1000
5/5 [==============================] - 2s 323ms/step - loss: 0.4860 - acc: 0.8761 - precision_1: 0.8651 - recall_1: 0.8889 - val_loss: 0.6096 - val_acc: 0.8774 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 38/1000
5/5 [==============================] - 2s 257ms/step - loss: 0.4714 - acc: 0.8878 - precision_1: 0.8616 - recall_1: 0.9234 - val_loss: 0.6068 - val_acc: 0.8774 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 39/1000
5/5 [==============================] - 2s 410ms/step - loss: 0.5570 - acc: 0.8238 - precision_1: 0.8498 - recall_1: 0.7920 - val_loss: 0.6026 - val_acc: 0.8774 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 40/1000
5/5 [==============================] - 2s 316ms/step - loss: 0.5372 - acc: 0

5/5 [==============================] - 2s 485ms/step - loss: 0.4744 - acc: 0.8624 - precision_1: 0.8877 - recall_1: 0.8367 - val_loss: 0.5648 - val_acc: 0.8774 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 72/1000
5/5 [==============================] - 2s 258ms/step - loss: 0.4093 - acc: 0.9099 - precision_1: 0.9051 - recall_1: 0.9156 - val_loss: 0.5731 - val_acc: 0.8774 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 73/1000
5/5 [==============================] - 2s 387ms/step - loss: 0.4746 - acc: 0.8587 - precision_1: 0.8777 - recall_1: 0.8449 - val_loss: 0.5690 - val_acc: 0.8774 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 74/1000
5/5 [==============================] - 2s 452ms/step - loss: 0.4108 - acc: 0.9003 - precision_1: 0.8934 - recall_1: 0.9142 - val_loss: 0.5533 - val_acc: 0.8774 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 75/1000
5/5 [==============================] - 2s 418ms/step - loss: 0.5248 - acc: 0

Epoch 107/1000
5/5 [==============================] - 2s 306ms/step - loss: 0.3845 - acc: 0.9028 - precision_1: 0.9100 - recall_1: 0.8949 - val_loss: 0.5058 - val_acc: 0.8654 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 108/1000
5/5 [==============================] - 2s 398ms/step - loss: 0.3930 - acc: 0.8967 - precision_1: 0.9048 - recall_1: 0.8975 - val_loss: 0.5094 - val_acc: 0.8662 - val_precision_1: 0.0962 - val_recall_1: 0.0109
Epoch 109/1000
5/5 [==============================] - 2s 260ms/step - loss: 0.3530 - acc: 0.9152 - precision_1: 0.9053 - recall_1: 0.9253 - val_loss: 0.5016 - val_acc: 0.8636 - val_precision_1: 0.1185 - val_recall_1: 0.0174
Epoch 110/1000
5/5 [==============================] - 2s 292ms/step - loss: 0.3637 - acc: 0.9060 - precision_1: 0.8996 - recall_1: 0.9126 - val_loss: 0.4968 - val_acc: 0.8613 - val_precision_1: 0.1250 - val_recall_1: 0.0218
Epoch 111/1000
5/5 [==============================] - 2s 388ms/step - loss: 0.3863 - acc: 0.8997

5/5 [==============================] - 2s 393ms/step - loss: 0.3869 - acc: 0.8921 - precision_1: 0.9094 - recall_1: 0.8782 - val_loss: 0.4852 - val_acc: 0.8559 - val_precision_1: 0.0914 - val_recall_1: 0.0196
Epoch 144/1000
5/5 [==============================] - 2s 368ms/step - loss: 0.3817 - acc: 0.8952 - precision_1: 0.9139 - recall_1: 0.8838 - val_loss: 0.4885 - val_acc: 0.8524 - val_precision_1: 0.0769 - val_recall_1: 0.0185
Epoch 145/1000
5/5 [==============================] - 2s 295ms/step - loss: 0.3386 - acc: 0.9111 - precision_1: 0.9217 - recall_1: 0.8989 - val_loss: 0.5095 - val_acc: 0.8424 - val_precision_1: 0.1571 - val_recall_1: 0.0654
Epoch 146/1000
5/5 [==============================] - 2s 364ms/step - loss: 0.3292 - acc: 0.9169 - precision_1: 0.9193 - recall_1: 0.9144 - val_loss: 0.5614 - val_acc: 0.8100 - val_precision_1: 0.1710 - val_recall_1: 0.1429
Epoch 147/1000
5/5 [==============================] - 2s 389ms/step - loss: 0.4441 - acc: 0.8565 - precision_1: 0.9017 

...

Epoch 837/1000
5/5 [==============================] - 2s 353ms/step - loss: 0.1714 - acc: 0.9549 - precision_1: 0.9561 - recall_1: 0.9535 - val_loss: 0.5316 - val_acc: 0.8509 - val_precision_1: 0.2408 - val_recall_1: 0.1003
Epoch 838/1000
5/5 [==============================] - 2s 346ms/step - loss: 0.1803 - acc: 0.9517 - precision_1: 0.9464 - recall_1: 0.9588 - val_loss: 0.5757 - val_acc: 0.8184 - val_precision_1: 0.1800 - val_recall_1: 0.1352
Epoch 839/1000
5/5 [==============================] - 2s 275ms/step - loss: 0.1884 - acc: 0.9534 - precision_1: 0.9485 - recall_1: 0.9586 - val_loss: 0.5194 - val_acc: 0.8242 - val_precision_1: 0.1616 - val_recall_1: 0.1036
Epoch 840/1000
5/5 [==============================] - 2s 401ms/step - loss: 0.2142 - acc: 0.9337 - precision_1: 0.9460 - recall_1: 0.9250 - val_loss: 0.5697 - val_acc: 0.7776 - val_precision_1: 0.1202 - val_recall_1: 0.1287
Epoch 841/1000
5/5 [==============================] - 2s 302ms/step - loss: 0.1950 - acc: 0.9397 - preci

5/5 [==============================] - 2s 371ms/step - loss: 0.1934 - acc: 0.9447 - precision_1: 0.9424 - recall_1: 0.9517 - val_loss: 0.5528 - val_acc: 0.7727 - val_precision_1: 0.2089 - val_recall_1: 0.3064
Epoch 874/1000
5/5 [==============================] - 2s 403ms/step - loss: 0.2027 - acc: 0.9325 - precision_1: 0.9473 - recall_1: 0.9174 - val_loss: 0.6022 - val_acc: 0.7155 - val_precision_1: 0.1722 - val_recall_1: 0.3468
Epoch 875/1000
5/5 [==============================] - 2s 355ms/step - loss: 0.1467 - acc: 0.9606 - precision_1: 0.9643 - recall_1: 0.9565 - val_loss: 0.6135 - val_acc: 0.6980 - val_precision_1: 0.1844 - val_recall_1: 0.4275
Epoch 876/1000
5/5 [==============================] - 2s 248ms/step - loss: 0.1479 - acc: 0.9641 - precision_1: 0.9673 - recall_1: 0.9604 - val_loss: 0.6292 - val_acc: 0.6822 - val_precision_1: 0.1793 - val_recall_1: 0.4449
Epoch 877/1000
5/5 [==============================] - 2s 275ms/step - loss: 0.2189 - acc: 0.9334 - precision_1: 0.9360 

Epoch 910/1000
5/5 [==============================] - 2s 373ms/step - loss: 0.1700 - acc: 0.9564 - precision_1: 0.9500 - recall_1: 0.9659 - val_loss: 0.4820 - val_acc: 0.7831 - val_precision_1: 0.2516 - val_recall_1: 0.3893
Epoch 911/1000
5/5 [==============================] - 2s 156ms/step - loss: 0.1613 - acc: 0.9534 - precision_1: 0.9507 - recall_1: 0.9550 - val_loss: 0.4796 - val_acc: 0.8140 - val_precision_1: 0.2587 - val_recall_1: 0.2770
Epoch 912/1000
5/5 [==============================] - 2s 380ms/step - loss: 0.2377 - acc: 0.9242 - precision_1: 0.9303 - recall_1: 0.9265 - val_loss: 0.6620 - val_acc: 0.6748 - val_precision_1: 0.2038 - val_recall_1: 0.5682
Epoch 913/1000
5/5 [==============================] - 2s 307ms/step - loss: 0.1669 - acc: 0.9473 - precision_1: 0.9466 - recall_1: 0.9472 - val_loss: 0.7202 - val_acc: 0.6612 - val_precision_1: 0.1922 - val_recall_1: 0.5507
Epoch 914/1000
5/5 [==============================] - 2s 267ms/step - loss: 0.1378 - acc: 0.9651 - preci

5/5 [==============================] - 2s 422ms/step - loss: 0.2099 - acc: 0.9332 - precision_1: 0.9313 - recall_1: 0.9473 - val_loss: 0.5472 - val_acc: 0.8361 - val_precision_1: 0.2480 - val_recall_1: 0.1658
Epoch 947/1000
5/5 [==============================] - 2s 276ms/step - loss: 0.1442 - acc: 0.9596 - precision_1: 0.9606 - recall_1: 0.9580 - val_loss: 0.4438 - val_acc: 0.8716 - val_precision_1: 0.4370 - val_recall_1: 0.1625
Epoch 948/1000
5/5 [==============================] - 2s 273ms/step - loss: 0.1333 - acc: 0.9657 - precision_1: 0.9721 - recall_1: 0.9587 - val_loss: 0.4219 - val_acc: 0.8708 - val_precision_1: 0.4482 - val_recall_1: 0.2312
Epoch 949/1000
5/5 [==============================] - 2s 289ms/step - loss: 0.1652 - acc: 0.9476 - precision_1: 0.9551 - recall_1: 0.9386 - val_loss: 0.4998 - val_acc: 0.8187 - val_precision_1: 0.2667 - val_recall_1: 0.2737
Epoch 950/1000
5/5 [==============================] - 2s 437ms/step - loss: 0.2254 - acc: 0.9285 - precision_1: 0.9322 

Epoch 983/1000
5/5 [==============================] - 2s 269ms/step - loss: 0.1270 - acc: 0.9689 - precision_1: 0.9671 - recall_1: 0.9705 - val_loss: 0.4717 - val_acc: 0.8355 - val_precision_1: 0.1663 - val_recall_1: 0.0851
Epoch 984/1000
5/5 [==============================] - 2s 437ms/step - loss: 0.1795 - acc: 0.9445 - precision_1: 0.9596 - recall_1: 0.9320 - val_loss: 0.5050 - val_acc: 0.8099 - val_precision_1: 0.1325 - val_recall_1: 0.0992
Epoch 985/1000
5/5 [==============================] - 2s 401ms/step - loss: 0.1550 - acc: 0.9532 - precision_1: 0.9581 - recall_1: 0.9477 - val_loss: 0.4977 - val_acc: 0.8097 - val_precision_1: 0.1966 - val_recall_1: 0.1788
Epoch 986/1000
5/5 [==============================] - 2s 411ms/step - loss: 0.1997 - acc: 0.9409 - precision_1: 0.9457 - recall_1: 0.9427 - val_loss: 0.4891 - val_acc: 0.8311 - val_precision_1: 0.2624 - val_recall_1: 0.2083
Epoch 987/1000
5/5 [==============================] - 2s 386ms/step - loss: 0.1903 - acc: 0.9393 - preci

In [20]:
%tensorboard --logdir logs/WorkingModelWithBiasGCN1

In [ ]:
model_loss = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done. Test loss: {}".format(model_loss))

1/1 [==============================] - 0s 473ms/step - loss: 3.8118 - acc: 0.5978 - precision_1: 0.2041 - recall_1: 0.9974
Done. Test loss: [3.8117854595184326, 0.5977643132209778, 0.20405036211013794, 0.9974315166473389]


In [ ]:
from sklearn.metrics import classification_report
print(len(dataset_te))
for test_data in dataset_te:
    X_te = test_data.x
    A_te = test_data.a
    y_te = test_data.y

    y_pred = model.predict([X_te, A_te, A_te], batch_size=X_te.shape[0])
    report = classification_report(y_te, (y_pred >= 0.5).astype(int))
    print('GCN Classification Report: \n {}'.format(report))

In [ ]:
from sklearn.metrics import classification_report
print(len(dataset_va))
for test_data in dataset_va:
    X_te = test_data.x
    A_te = test_data.a
    y_te = test_data.y

    y_pred = model.predict([X_te, A_te, A_te], batch_size=X_te.shape[0])
    report = classification_report(y_te, (y_pred >= 0.5).astype(int))
    print('GCN Classification Report: \n {}'.format(report))

In [ ]:
from sklearn.metrics import classification_report
print(len(dataset_tr))
for test_data in dataset_tr:
    X_te = test_data.x
    A_te = test_data.a
    y_te = test_data.y

    y_pred = model.predict([X_te, A_te, A_te], batch_size=X_te.shape[0])
    report = classification_report(y_te, (y_pred >= 0.5).astype(int))
    print('GCN Classification Report: \n {}'.format(report))